In [27]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import re
import random

In [28]:
from datetime import datetime
pd.options.mode.chained_assignment = None  # default='warn'

# Manipulating Data

## Method Definition + Data Setup

In [29]:
#CREATING DATAFRAMES
df_ridership = pd.read_csv(r"Z:\Data\RIDERSHIP\RidershipData.csv")
df_feedback = pd.read_csv(r"Z:\Data\RIDER FEEDBACK\RidershipFeedback.csv")

In [30]:
df_feedback['Date'] = pd.to_datetime(df_feedback['Date'])
df_ridership['Day'] = pd.to_datetime(df_ridership['Day'])

In [31]:
def date_to_semester(df):
    df['Semester'] = df['Date'].dt.month.map({
        1: 'Spring', 2: 'Spring', 3: 'Spring', 4: 'Spring', 5: 'Spring',
        6: 'Summer', 7: 'Summer',
        8: 'Fall', 9: 'Fall', 10: 'Fall', 11: 'Fall', 12: 'Fall'
    })
    df['Year'] = df['Date'].dt.year
    df['Semester and Year'] = df['Semester'] + ' ' + df['Year'].astype(str)
    return df

df_feedback = date_to_semester(df_feedback)

## Creating Table of Vehicle Capacity Reached + Filled Capacity Complaints

### Ridership Table of Vehicle Capacity Being Reached

In [32]:
df_veh_cap = df_ridership[(df_ridership['Riders Cumulative'] >= df_ridership['Vehicle Capacity']) & (df_ridership['Riders On'] > 0)]

In [33]:
# Group the filtered dataframe by 'Day' and 'Route' and count the occurrences
df_veh_cap = df_veh_cap.groupby(['Day', 'Route']).size()

In [34]:
# Reset the index of the grouped dataframe to make it a flat table
df_veh_cap = df_veh_cap.reset_index(name='Count of Vehicle Capacity Reached')
df_veh_cap.head(10)

,Day,Route,Count of Vehicle Capacity Reached
0,2023-08-31,Waltham Shuttle,10
1,2023-09-01,Campus Shuttle,20
2,2023-09-01,Waltham Shuttle,2
3,2023-09-02,Boston Cambridge Shuttle,3
4,2023-09-02,Combo Route,15
5,2023-09-03,Boston Cambridge Shuttle,3
6,2023-09-04,Combo Route,49
7,2023-09-05,Campus Shuttle,41
8,2023-09-05,Waltham Shuttle,4
9,2023-09-06,Campus Shuttle,30


### Complaints Table of Filled Bus

In [35]:
df_lb = df_feedback[df_feedback['Issue Type'] == 'Leave Behinds']
df_lb = df_lb.groupby('Date').size().reset_index(name='Count of Leave Behinds')
df_lb.head(10)

,Date,Count of Leave Behinds
0,2023-08-31,2
1,2023-09-05,1
2,2023-09-12,2
3,2023-09-14,1
4,2023-10-10,1
5,2023-10-14,1
6,2023-10-30,1
7,2023-11-06,2
8,2023-11-14,1
9,2024-01-21,1


### Combination of Tables

Given the dataframes df_veh_cap and df_lb, perform a left-join on df_veh_cap where the join attribute should be the 'Date' column in df_lb and 'Day in df_veh_cap. If there is no date value in df_lb that df_veh_cap has, put the value for the 'Count of Leave Behinds' down as 0.

In [36]:
merged_df_fullcap = pd.merge(df_veh_cap, df_lb, how='left', left_on='Day', right_on='Date')

# Fill NaN values in 'Count of Leave Behinds' column with 0
merged_df_fullcap['Count of Leave Behinds'].fillna(0, inplace=True)

# Drop the redundant 'Date' column
merged_df_fullcap.drop('Date', axis=1, inplace=True)

merged_df_fullcap.head(10)

,Day,Route,Count of Vehicle Capacity Reached,Count of Leave Behinds
0,2023-08-31,Waltham Shuttle,10,2.0
1,2023-09-01,Campus Shuttle,20,0.0
2,2023-09-01,Waltham Shuttle,2,0.0
3,2023-09-02,Boston Cambridge Shuttle,3,0.0
4,2023-09-02,Combo Route,15,0.0
5,2023-09-03,Boston Cambridge Shuttle,3,0.0
6,2023-09-04,Combo Route,49,0.0
7,2023-09-05,Campus Shuttle,41,1.0
8,2023-09-05,Waltham Shuttle,4,1.0
9,2023-09-06,Campus Shuttle,30,0.0


## Creating Table of Ridership + Feedback by Month/Year

In [39]:
# Group by 'Day' and sum all columns
df_daily_ridership = df_ridership.groupby('Day')['Riders On'].sum().reset_index()

In [40]:
df_daily_complaints = df_feedback.groupby(df_feedback['Date'].dt.date).size().reset_index(name='Number of Complaints')

In [41]:
df_daily_complaints['Date'] = pd.to_datetime(df_daily_complaints['Date'])

df_combined_totals = pd.merge(df_daily_ridership, df_daily_complaints, left_on='Day', right_on='Date')

# Drop the redundant 'Date' column
df_combined_totals.drop('Date', axis=1, inplace=True)

df_combined_totals.head(10)

,Day,Riders On,Number of Complaints
0,2023-08-31,474,4
1,2023-09-01,427,1
2,2023-09-02,552,1
3,2023-09-04,150,1
4,2023-09-05,941,1
5,2023-09-07,1110,1
6,2023-09-08,762,2
7,2023-09-10,369,1
8,2023-09-11,558,2
9,2023-09-12,716,3


In [42]:
# Group by Month and Year and sum the values of 'Riders On' and 'Number of Complaints' columns
df_combined_totals = df_combined_totals.groupby(df_combined_totals['Day'].dt.to_period('M')).agg({'Riders On': 'sum', 'Number of Complaints': 'sum'})

# Reset the index to make 'Day' a regular column
df_combined_totals.reset_index(inplace=True)

In [43]:
print(df_combined_totals)

       Day  Riders On  Number of Complaints
0  2023-08        474                     4
1  2023-09      12102                    27
2  2023-10      13093                    24
3  2023-11       8082                    19
4  2023-12       3384                    24
5  2024-01       7614                    15
6  2024-02       3654                     6
7  2024-03      11397                    21
8  2024-04       1739                     5


## Creating Table of Vehicle Capacity + Leave Behinds

# Dash App

In [44]:
# Convert 'Date' column to datetime format
if isinstance(df_ridership['Day'][0], str):
    df_ridership['Day'] = pd.to_datetime(df_ridership['Day'])
min_date = min(option['value'] for option in [{'label': str(date), 'value': str(date)} for date in df_feedback['Date'].dt.to_period('M').unique()])


In [45]:
# Create the Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Feedback Dashboard"),

    # Plotly chart
    dcc.Graph(id='ridership1-chart'),
    dcc.Graph(id='ridership-chart'),
    
    # DROPDOWNS OF TIME
    html.Div([
        # Dropdown for selecting month and year
        html.Div([
            html.Label("Select Month and Year:"),
            dcc.Dropdown(
                id='month-year-dropdown',
                options=[{'label': str(date), 'value': str(date)} for date in df_feedback['Date'].dt.to_period('M').unique()],
                value=min_date,
            ),
        ], style={'width': '75%', 'display': 'inline-block', 'padding': '0 10px'}),
        
        # Dropdown for selecting semester and year
        html.Div([
            html.Div([
                html.Div([
                    html.Label("Select Semester and Year:"),
                    dcc.Dropdown(
                        id='semester-year-dropdown',
                        options=[{'label': semester_year, 'value': semester_year} for semester_year in df_feedback['Semester and Year'].unique()],
                        value=df_feedback['Semester and Year'].iloc[0],  # Set initial value to the first semester and year in the DataFrame
                    ),
                    
                ], style={'width':'50%'}),
                html.Div([
                    html.Label("Select Category:"),
                    dcc.Dropdown(
                        id='category-pie-dropdown',
                        options=[{'label': 'All', 'value': 'All'}] + [{'label': category, 'value': category} for category in df_feedback['Category'].unique()],# fill in here to be 'All' and all 'Category' values ,
                        value='All'
                    )
                ], style={'width':'50%'})
            ], style={'display':'flex'}),
        ], style={'width': '75%', 'display': 'inline-block', 'padding': '0 10px'}),
        
    ], style={'display':'flex'}),

    # VISIUALISATIONS
    html.Div([
        # Bar graph for counts of categories based on selected month and year
        html.Div([
            dcc.Graph(id='category-counts-graph'),
            
        ], style={'width':'50%'}),
        
        # PIE CHART HERE
        html.Div([
            dcc.Graph(id='sunburst-chart'),
        ], style={'width':'50%'}),
    ], style={'display':'flex'}),
    
    # Dropdown for selecting category
    html.Label("Select Category:"),
    dcc.Dropdown(
        id='category-dropdown',
        options=[{'label': category, 'value': category} for category in df_feedback['Category'].unique()],
        value=df_feedback['Category'].iloc[0]
    ),
    
    # Bar graph for counts/sum of rows corresponding to selected category and month/year
    dcc.Graph(id='monthly-category-counts-graph')
    
    # html.H2("Analysis on Vehicle Capacity"),
    # dcc.Graph(id='capacity-avg-chart'),
    
], style={'font-family': 'Segoe UI'})

### Bar Chart: Feedback Count by Selected Month and Categories

In [46]:
@app.callback(
    Output('category-counts-graph', 'figure'),
    [Input('month-year-dropdown', 'value')]
)
def update_category_counts(month_year):
    filtered_data = df_feedback[df_feedback['Date'].dt.to_period('M') == pd.Period(month_year)]
    category_counts = filtered_data['Category'].value_counts().reset_index()
    category_counts.columns = ['Category', 'Count']  # Rename the columns
    fig = px.bar(category_counts, x='Category', y='Count', labels={'x': 'Category', 'y': 'Count'},
                color_discrete_sequence=px.colors.qualitative.Dark2)
    fig.update_layout(title='Category Counts for {}'.format(month_year))
    return fig


### Pie Chart: Category Distribution by Semester

Given the above code, change the sunburst chart instead so that it is a static pie chart that changes based on the category-pie-dropdown and semester-year-dropdown.
For the category-pie-dropdown, when 'All' is selected it should show the distribution of the 'Category' values. When any other value is selected, it should find all rows whose category is in category-pie-dropdown and then show all 'Issue Type' distribution for that selected category.

In [47]:
@app.callback(
    Output('sunburst-chart', 'figure'),
    [Input('semester-year-dropdown', 'value'),
     Input('category-pie-dropdown', 'value')]
)
def update_pie_chart(selected_semester_year, selected_category):
    if selected_category == 'All':
        # If 'All' is selected, show distribution of 'Category' values
        filtered_df = df_feedback[df_feedback['Semester and Year'] == selected_semester_year]
        category_counts = filtered_df['Category'].value_counts()
        labels = category_counts.index.tolist()
        values = category_counts.values.tolist()
        title = f'Distribution of Categories for {selected_semester_year}'
    else:
        # Show 'Issue Type' distribution for the selected category
        filtered_df = df_feedback[(df_feedback['Semester and Year'] == selected_semester_year) & 
                                  (df_feedback['Category'] == selected_category)]
        issue_type_counts = filtered_df['Issue Type'].value_counts()
        labels = issue_type_counts.index.tolist()
        values = issue_type_counts.values.tolist()
        title = f'Distribution of Issue Types for {selected_category} ({selected_semester_year})'

    # Create pie chart using Plotly Express
    fig = px.pie(names=labels, 
                    values=values, 
                    color_discrete_sequence=px.colors.qualitative.Dark2, 
                    title=title)
    
    fig.update_traces(textinfo='label+percent')

    return fig

### Bar Chart: Selected Category over Time (Month/Year)

In [48]:
@app.callback(
    Output('monthly-category-counts-graph', 'figure'),
    [Input('category-dropdown', 'value')]
)
def update_monthly_category_counts(category):
    # Filter data for selected category
    category_data = df_feedback[df_feedback['Category'] == category]
    # Group by both Date and Issue Type and count occurrences
    monthly_counts = category_data.groupby([category_data['Date'].dt.to_period('M'), 'Issue Type']).size().unstack(fill_value=0)
    # Create stacked bar chart with custom color palette
    fig = px.bar(monthly_counts, x=monthly_counts.index.strftime('%B %Y'), y=monthly_counts.columns, 
                 labels={'x': 'Month and Year', 'y': 'Count'}, barmode='stack',
                 color_discrete_sequence=px.colors.qualitative.Dark2)  # Example of using Set3 color palette
    fig.update_layout(title='Monthly Counts for Category: {}'.format(category))
    return fig


### Bar Chart: Subplot Bar Chart Trends

Given the code for the subplot bar chart ridership-chart, replace it with a combined bar and line chart with 2 y-axes, where the bar chart the total ridership and the y-axis on the left counts is the 'Riders On'. Have the line chart be represented by the Feedback total, where the y-axis should be the percentage of number of complaints of the ridership calculated by 'Number of Complaints' / 'Riders On'). The x-axis should remain the same 'Day'.

In [49]:
# Update the ridership chart with the required subplots
@app.callback(
    Output('ridership1-chart', 'figure'),
    [Input('month-year-dropdown', 'value')]  # Assuming the dropdown is removed
)
def update_ridership_chart(_):
    # Use the entire df_combined_totals DataFrame
    filtered_df = df_combined_totals
    
    # Convert PeriodIndex to string representation
    filtered_df['Day'] = filtered_df['Day'].astype(str)
    
    # Create the figure
    fig = make_subplots(rows=2, cols=1, 
                        shared_xaxes=True,
                        vertical_spacing=0.03,
                        specs=[[{"type": "bar"}],
                               [{"type": "bar"}]]
                       )

    # Add subplot 1 - Total Ridership
    fig.add_trace(
        go.Bar(
            x=filtered_df['Day'],
            y=filtered_df['Riders On'],
            name='Total Ridership',
            text=filtered_df['Riders On'],  # Display count on top of each bar
            textposition='auto',  # Automatically position the text
    ), row=1, col=1)

    # Add subplot 2 - Feedback total
    fig.add_trace(
        go.Bar(
            x=filtered_df['Day'],
            y=filtered_df['Number of Complaints'],
            name='Feedback Total',
            text=filtered_df['Number of Complaints'],  # Display count on top of each bar
            textposition='auto',  # Automatically position the text
    ), row=2, col=1)

    # Update layout
    fig.update_layout(
        height=600,
        title_text="Ridership and Feedback Overlap",
        legend=dict(
            orientation="h",
            yanchor="top",
            y=1.02,
            xanchor="right",
            x=1
        ),
        colorway=[px.colors.qualitative.Dark2[1], px.colors.qualitative.Dark2[0]]  # Change the color sequence
    )

    # Convert Figure object to dictionary
    fig_dict = fig.to_dict()

    return fig_dict


In [50]:
# Update the ridership chart with a combined bar and line chart
@app.callback(
    Output('ridership-chart', 'figure'),
    [Input('month-year-dropdown', 'value')]
)
def update_ridership_chart(_):
    # Use the entire df_combined DataFrame
    filtered_df = df_combined_totals
    
    # Convert PeriodIndex to string representation
    filtered_df['Day'] = filtered_df['Day'].astype(str)
    
    # Calculate percentage of complaints relative to ridership
    filtered_df['Complaints Percentage'] = (filtered_df['Number of Complaints'] / filtered_df['Riders On']) 

    # Create the figure
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Color definition
    color1 = px.colors.qualitative.Dark2[5]
    color2 = px.colors.qualitative.Dark2[0]

    # Add bar chart for total ridership
    fig.add_trace(
        go.Bar(
            x=filtered_df['Day'],
            y=filtered_df['Riders On'],
            name='Total Ridership',
            text=filtered_df['Riders On'],  # Display count on top of each bar
            textposition='auto',
            marker_color=color1
        )
    )

    # Add line chart for complaints percentage
    fig.add_trace(
        go.Scatter(
            x=filtered_df['Day'],
            y=filtered_df['Complaints Percentage'],
            name='Feedback Percentage(%)',
            mode='lines+markers+text',
            text=[f"{val * 100:.2f}%" for val in filtered_df['Complaints Percentage']],  # Convert to percentage and format to two decimal places
            line=dict(color=color2),
            textposition='top center',
            showlegend=True
        ),
        secondary_y=True
    )
    
    # Update layout
    fig.update_layout(
        title_text="Ridership and Feedback Overlap",
        xaxis=dict(title='Day'),
        yaxis=dict(title='Total Ridership'),
        yaxis2=dict(title='Feedback-to-Ridership Percentage (%)', overlaying='y', side='right', showgrid=False, tickformat='%'),
        legend=dict(
            orientation="h",
            yanchor="top",
            y=1.02,
            xanchor="right",
            x=1
        ),
        colorway=[color1, color2]  # Change the color sequence
    )

    return fig

### Run the Dashboard on local server

In [52]:
# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, port=1001)